In [ ]:
import pandas as pd
import numpy as np
import pymorphy2
from navec import Navec
from sklearn.metrics.pairwise import cosine_similarity
from pymorphy2 import MorphAnalyzer
from razdel import tokenize
path = 'data/navec_hudlit_v1_12B_500K_300d_100q.tar'
navec = Navec.load(path)
morph = MorphAnalyzer()
popularity = pd.read_csv('data/csv/query_popularity.csv')
chunksize = 10 ** 4
filename = 'data/csv/search_history.csv'
data = pd.read_csv(filename, chunksize=chunksize)
data = data.get_chunk(3500000)

In [ ]:
#preprocess data from search_history
unique_uq = data['UQ'].unique()
x = []
for e in unique_uq:
    if type(e) == str:
        x.append(e)
x = np.array(x)
for e in ["/",'!','@',"#","$","%","&","*","http"]:
    x = np.delete(x,np.flatnonzero(np.core.defchararray.find(x,e)!=-1))

In [ ]:
#preprocess data from query_popularity
y = []
unique_pop = popularity['query'].unique()
for e in unique_pop:
    if type(e) == str:
        y.append(e)
y = np.array(y)
for e in ["/",'!','@',"#","$","%","&","*","http"]:
    y = np.delete(y,np.flatnonzero(np.core.defchararray.find(y,e)!=-1))

In [ ]:
#take 5k and 7r of search queries respectively
test = x[:5000]
# f function normalize words in the sentences 
f = lambda el: ' '.join([morph.parse(e.text)[0].normal_form for e in list(tokenize(el))])
vf = np.vectorize(f)
preprocessed_x = vf(test)
preprocessed_y = vf(y[:5000])

In [ ]:
def get_cosine_similarity(str_x,str_y):
    f = lambda el: ' '.join([morph.parse(e.text)[0].normal_form for e in list(tokenize(el))])
    try:
        x_x = navec['<pad>']
        for e in f(str_x).split(' '):
            x_x+=navec[e]
        y_y = navec['<pad>']
        for e in f(str_y).split(' '):
            y_y+=navec[e]
    except KeyError:
        pass
    return cosine_similarity([x_x],[y_y])[0][0]

In [ ]:
get_cosine_similarity('ботинки женские мембрана','кроссовки мужские мембрана')

In [ ]:
dt_x = pd.DataFrame(columns=['UQ'],data=preprocessed_x)
dt_y = pd.DataFrame(columns=['UQ'],data=preprocessed_y)

threshold = 0.6 #less than threshold is dropped

big = [[],[]]

for x_s in range(50):#change max index of search_history query
    temp = []
    temp1 = []
    for y_s in range(len(dt_y['UQ'])):#change here index of popularity query
        try:
            answ = navec['<pad>']
            for e in dt_x['UQ'][x_s].split(' '):
                answ+=navec[e]
            answ2 = navec['<pad>']
            for e in dt_y['UQ'][y_s].split(' '):
                answ2+=navec[e]
            if(cosine_similarity([answ],[answ2])>threshold):
                temp.append([dt_x['UQ'][x_s],dt_y['UQ'][y_s]])
                temp1.append(cosine_similarity([answ],[answ2]))
        except KeyError:
            pass
    big[0].append(temp)
    big[1].append(temp1)

In [ ]:
#code to visualize
for i in range(len(big[0])):
    print(i)
    c = 0
    for e in sorted(big[1][i])[::-1]:
        if c == 10:
            break
        else:
            print(big[0][i][big[1][i].index(e)],e)
            c+=1